# Coffee Shop EDA and Seasonal Drink Recommender

This notebook loads sales, survey (GACTT), and Yelp sentiment data, performs basic EDA, and builds a simple rule-based recommender.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

DATA_PATH = "../data/"
OUTPUT_PATH = "../outputs/"
os.makedirs(OUTPUT_PATH, exist_ok=True)

sales = pd.read_csv(DATA_PATH + "merged_sales_product.csv", parse_dates=["transaction_date"])
outlets = pd.read_csv(DATA_PATH + "sales_outlet_clean.csv")
products = pd.read_csv(DATA_PATH + "product_clean.csv")
customers = pd.read_csv(DATA_PATH + "customer_clean.csv")
survey = pd.read_csv(DATA_PATH + "GACTT_RESULTS_ANONYMIZED_v2.csv")
yelp_sent = pd.read_csv(DATA_PATH + "sentiments_by_shop.csv")

sales.head()

In [ ]:
# feature engineering
sales["date"] = sales["transaction_date"].dt.date
sales["dayofweek"] = sales["transaction_date"].dt.day_name()
sales["month"] = sales["transaction_date"].dt.month

daily_rev = (sales.groupby("date")["line_item_amount"]
             .sum()
             .reset_index()
             .rename(columns={"line_item_amount": "daily_revenue"}))
daily_rev.head()

In [ ]:
# top products
top_products = (sales.groupby("product")["line_item_amount"]
                .sum()
                .reset_index()
                .sort_values("line_item_amount", ascending=False))
top_products.head(10)

In [ ]:
# survey parsing based on provided columns

fav_col_1 = "What is your favorite coffee drink?"
fav_col_2 = "Please specify what your favorite coffee drink is"

survey_favs = []
if fav_col_1 in survey.columns:
    survey_favs.append(survey[fav_col_1].dropna().str.strip().str.title())
if fav_col_2 in survey.columns:
    survey_favs.append(survey[fav_col_2].dropna().str.strip().str.title())

if survey_favs:
    survey_favs = pd.concat(survey_favs)
    fav_counts = (survey_favs.value_counts()
                  .reset_index()
                  .rename(columns={"index": "drink_name", 0: "mentions"}))
else:
    fav_counts = pd.DataFrame(columns=["drink_name", "mentions"])

fav_counts.head(10)

In [ ]:
# normalize drink names to map to products
def normalize_drink_name(s):
    if pd.isna(s):
        return None
    s = s.lower().strip()
    s = s.replace("iced ", "")
    s = s.replace("ice ", "")
    s = s.replace("latte ", "latte")
    return s

reco = top_products.copy()
reco.rename(columns={"line_item_amount": "sales_revenue"}, inplace=True)
reco["product_norm"] = reco["product"].apply(normalize_drink_name)

fav_counts["drink_norm"] = fav_counts["drink_name"].apply(normalize_drink_name)

reco = reco.merge(
    fav_counts[["drink_norm", "mentions"]],
    left_on="product_norm",
    right_on="drink_norm",
    how="left"
)

def minmax(s):
    return (s - s.min()) / (s.max() - s.min()) if s.max() != s.min() else 1.0

reco["sales_score"] = minmax(reco["sales_revenue"])
reco["survey_popularity_score"] = reco["mentions"].fillna(0)
if reco["survey_popularity_score"].max() > 0:
    reco["survey_popularity_score"] = reco["survey_popularity_score"] / reco["survey_popularity_score"].max()
else:
    reco["survey_popularity_score"] = 0

# yelp sentiment (optional)
if not yelp_sent.empty and "avg_sentiment" in yelp_sent.columns:
    sentiment_default = yelp_sent["avg_sentiment"].mean()
else:
    sentiment_default = 0.1
reco["sentiment_score"] = sentiment_default

reco["final_score"] = (
    0.55 * reco["sales_score"] +
    0.30 * reco["survey_popularity_score"] +
    0.10 * reco["sentiment_score"] +
    0.05 * 1  # placeholder
)

reco_sorted = reco.sort_values("final_score", ascending=False)
reco_sorted.head(15)

In [ ]:
# export
daily_rev.to_csv(OUTPUT_PATH + "daily_revenue.csv", index=False)
top_products.to_csv(OUTPUT_PATH + "top_products.csv", index=False)
reco_sorted.head(50).to_csv(OUTPUT_PATH + "drink_recommendations.csv", index=False)
"exports written" 

## Visualizations

In [ ]:
import matplotlib.pyplot as plt

# daily revenue plot
plt.figure()
plt.plot(daily_rev["date"], daily_rev["daily_revenue"])
plt.title("Daily Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# top 10 products bar
top10 = top_products.head(10)
plt.figure()
plt.bar(top10["product"], top10["line_item_amount"])
plt.title("Top 10 Products by Revenue")
plt.xticks(rotation=60, ha="right")
plt.tight_layout()
plt.show()

# revenue by day of week
dow_rev = (sales.groupby("dayofweek")["line_item_amount"]
           .sum()
           .reindex(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])
           .reset_index())
plt.figure()
plt.bar(dow_rev["dayofweek"], dow_rev["line_item_amount"])
plt.title("Revenue by Day of Week")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()